##### Bibliotecas

In [1]:
import pandas as pd
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

##### Funções

In [2]:
def extrair_partidas(url, data):
    service = Service() # Usada para iniciar uma instância do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized') # Define a preferência de browser para o Chrome
    driver = webdriver.Chrome(service=service, options=options) # Inicia a instância
    time.sleep(5)
    driver.get(url)

    time.sleep(10)
    partidas = driver.find_elements(By.CLASS_NAME, 'tournament-matches-row')
    lista_partidas = []

    # Pega dados
    for i in range(len(partidas)):
        partida = {}
        partida['data'] = data
        partida['time_casa'] = partidas[i].text.split('\n')[4]
        partida['player_casa'] = partidas[i].text.split('\n')[5]
        partida['gols_casa'] = int(partidas[i].text.split('\n')[6])
        partida['gols_fora'] = int(partidas[i].text.split('\n')[8])
        partida['player_fora'] = partidas[i].text.split('\n')[11]
        partida['time_fora'] = partidas[i].text.split('\n')[10]
        partida['total_gols'] = partida['gols_casa'] + partida['gols_fora']
        
        # Gols
        if partida['total_gols'] > 3.5:
            partida['+ 0.5'] = 'Sim'
            partida['+ 1.5'] = 'Sim'
            partida['+ 2.5'] = 'Sim'
            partida['+ 3.5'] = 'Sim'
        else:
            if partida['total_gols'] > 2.5:
                partida['+ 0.5'] = 'Sim'
                partida['+ 1.5'] = 'Sim'
                partida['+ 2.5'] = 'Sim'
                partida['+ 3.5'] = 'Não'
            else:
                if partida['total_gols'] > 1.5:
                    partida['+ 0.5'] = 'Sim'
                    partida['+ 1.5'] = 'Sim'
                    partida['+ 2.5'] = 'Não'
                    partida['+ 3.5'] = 'Não'
                else:
                    if partida['total_gols'] > 0.5:
                        partida['+ 0.5'] = 'Sim'
                        partida['+ 1.5'] = 'Não'
                        partida['+ 2.5'] = 'Não'
                        partida['+ 3.5'] = 'Não'
                    else:
                        partida['+ 0.5'] = 'Não'
                        partida['+ 1.5'] = 'Não'
                        partida['+ 2.5'] = 'Não'
                        partida['+ 3.5'] = 'Não'

        
        # Vitoria
        if partida['gols_casa'] > partida['gols_fora']:
            partida['vencedor'] = 'Casa'
        elif partida['gols_casa'] < partida['gols_fora']:
            partida['vencedor'] = 'Fora'
        else:
            partida['vencedor'] = 'Empate'
        


        lista_partidas.append(partida)
        
    driver.close()
    driver.quit()
    return lista_partidas

def extrair_torneios(url):
    service = Service() # Usada para iniciar uma instância do Chrome
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized') # Define a preferência de browser para o Chrome
    driver = webdriver.Chrome(service=service, options=options) # Inicia a instância
    driver.get(url)
    time.sleep(10)
    links = driver.find_elements(By.CLASS_NAME, 'icon-link')

    lista_torneios = []
    for i in range(len(links)):
        lista_torneios.append(links[i].get_attribute('href'))

    return lista_torneios

def salvar(dados):
    dados = dados
    for i in range(len(dados)):
        with open('base_jogos.csv', 'a', newline='', encoding='utf-8') as arquivo:
            escrever = csv.writer(arquivo)
            escrever.writerow([dados[i]['data'], dados[i]['time_casa'], dados[i]['player_casa'], 
            dados[i]['gols_casa'], dados[i]['gols_fora'], dados[i]['player_fora'], dados[i]['time_fora'], 
            dados[i]['total_gols'], dados[i]['+ 0.5'], dados[i]['+ 1.5'],dados[i]['+ 2.5'],
            dados[i]['+ 3.5'], dados[i]['vencedor']])
    


##### Preparação

In [5]:
# Cria uma lista de datas
list_datas = [(datetime(2022, 1, 15) + timedelta(days=n)).strftime('%Y-%m-%d') 
              for n in range((datetime(2022, 12, 31) - datetime(2022, 1, 1)).days + 1)]

# Cria um lista de URLs com as datas
pags_urls = []
for i in range(len(list_datas)):
    url = f"https://football.esportsbattle.com/en/schedule?page=1&dateFrom={list_datas[i].split('-')[0]}%2F{list_datas[i].split('-')[1]}%2F{list_datas[i].split('-')[2]}+08%3A00&dateTo={list_datas[i].split('-')[0]}%2F{list_datas[i].split('-')[1]}%2F{list_datas[i].split('-')[2]}+23%3A59"
    pags_urls.append(url)    

##### Inicialização das Funções

In [6]:
cont = 0
for url in pags_urls:
    torneios = extrair_torneios(url)
    for torneio in torneios:
        partidas = extrair_partidas(torneio, list_datas[cont])
        salvar(partidas)
    cont += 1
time.sleep(10)    

WebDriverException: Message: no such execution context
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF651127012+3522402]
	(No symbol) [0x00007FF650D48352]
	(No symbol) [0x00007FF650BF596D]
	(No symbol) [0x00007FF650BE33A6]
	(No symbol) [0x00007FF650BE13D1]
	(No symbol) [0x00007FF650BE1E3F]
	(No symbol) [0x00007FF650BEFB40]
	(No symbol) [0x00007FF650C01D43]
	(No symbol) [0x00007FF650C0618A]
	(No symbol) [0x00007FF650BE2400]
	(No symbol) [0x00007FF650C01913]
	(No symbol) [0x00007FF650C7BC7A]
	(No symbol) [0x00007FF650C5EE03]
	(No symbol) [0x00007FF650C2F4D4]
	(No symbol) [0x00007FF650C305F1]
	GetHandleVerifier [0x00007FF651159B9D+3730157]
	GetHandleVerifier [0x00007FF6511AF02D+4079485]
	GetHandleVerifier [0x00007FF6511A75D3+4048163]
	GetHandleVerifier [0x00007FF650E7A649+718233]
	(No symbol) [0x00007FF650D54A3F]
	(No symbol) [0x00007FF650D4FA94]
	(No symbol) [0x00007FF650D4FBC2]
	(No symbol) [0x00007FF650D3F2E4]
	BaseThreadInitThunk [0x00007FFCA8347344+2